# 12 - Projects // House Pricing

https://github.com/ad3rinto/data-science-rune/blob/main/12%20-%20Project%20-%20House%20Price.ipynb

In [ ]:
!pip install mlxtend

In [ ]:
import pandas as pd
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [ ]:
data = pd.read_parquet('https://github.com/ad3rinto/data-science-rune/raw/refs/heads/main/files/house_sales.parquet')
data.head()

,MSSubClass,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LandSlope,OverallQual,OverallCond,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,MiscVal,MoSold,YrSold,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,65.0,8450,1,3,3,3,2,7,5,...,61,0,0,0,0,NaN,0,2,2008,208500
2,20,80.0,9600,1,3,3,3,2,6,8,...,0,0,0,0,0,NaN,0,5,2007,181500
3,60,68.0,11250,1,2,3,3,2,7,5,...,42,0,0,0,0,NaN,0,9,2008,223500
4,70,60.0,9550,1,2,3,3,2,7,5,...,35,272,0,0,0,NaN,0,2,2006,140000
5,60,84.0,14260,1,2,3,3,2,8,5,...,84,0,0,0,0,NaN,0,12,2008,250000


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1460 entries, 1 to 1460
Data columns (total 56 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   LotFrontage    1201 non-null   float64
 2   LotArea        1460 non-null   int64  
 3   Street         1460 non-null   int64  
 4   LotShape       1460 non-null   int64  
 5   LandContour    1460 non-null   int64  
 6   Utilities      1460 non-null   int64  
 7   LandSlope      1460 non-null   int64  
 8   OverallQual    1460 non-null   int64  
 9   OverallCond    1460 non-null   int64  
 10  YearBuilt      1460 non-null   int64  
 11  YearRemodAdd   1460 non-null   int64  
 12  MasVnrArea     1452 non-null   float64
 13  ExterQual      1460 non-null   int64  
 14  ExterCond      1460 non-null   int64  
 15  BsmtQual       1423 non-null   float64
 16  BsmtCond       1423 non-null   float64
 17  BsmtExposure   1422 non-null   float64
 18  BsmtFinType1 

In [ ]:
data = data.drop('PoolQC', axis=1)

In [ ]:
data = data.fillna(-1)

**Analyze**

---



Quasi constant features

In [ ]:
sel = VarianceThreshold(threshold=0.01)
sel.fit(data)

VarianceThreshold(threshold=0.01)

In [ ]:
len(sel.get_feature_names_out())

53

In [ ]:
quasi_features = [col for col in data.columns if col not in sel.get_feature_names_out()]

In [ ]:
quasi_features

['Street', 'Utilities']

Correlated features

In [ ]:
corr_matrix = data.corr()

In [ ]:
corr_features = [feature for feature in corr_matrix.columns if (corr_matrix[feature].iloc[:corr_matrix.columns.get_loc(feature)] >0.8).any()]

In [ ]:
corr_features

['1stFlrSF',
 'TotRmsAbvGrd',
 'FireplaceQu',
 'GarageArea',
 'GarageQual',
 'GarageCond']

Prepare training and test set

In [ ]:
X = data.drop(['SalePrice'] + quasi_features + corr_features, axis=1)
y = data['SalePrice']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

Finding 10 best features for LinearRegressionmodel

In [ ]:
sfs = SFS(LinearRegression(), k_features=10, verbose=2)

In [ ]:
sfs.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.4s

[2024-05-15 07:20:53] Features: 1/10 -- score: 0.6126893627288714[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.4s

[2024-05-15 07:20:53] Features: 2/10 -- score: 0.7190976519156257[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.4s

[2024-05-15 07:20:54] Features: 3/10 -- score: 0.7711947823299818[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.4s

[2024-05-15 07:20:54] Features: 4/10 -- score: 0.7940453445923799[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.4s

[2024-05-15 07:20:55] Features: 5/10 -- score: 0.8078796820663863[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.4s

[2024-05-15 07:20:55] Features: 6/10 -- score: 0.817190522837052[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.4s
[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    0.4s

[2024-05-15 07:20:56] Features: 7/10 -- score: 0.8248996546748832
[2024-05-15 07:20:56] Features: 8/10 -- score: 0.8

SequentialFeatureSelector(estimator=LinearRegression(), k_features=(10, 10),
                          scoring='r2', verbose=2)

In [ ]:
print('The best features are :', sfs.k_feature_idx_)

The best features are : (0, 2, 6, 10, 11, 15, 17, 26, 33, 36)


Test the result

In [ ]:
lin = LinearRegression()
lin.fit(X_train, y_train)
y_pred = lin.predict(X_test)
r2_score(y_test, y_pred)

0.7110172392141719

In [ ]:
lin = LinearRegression()

columns = X_train.columns[list(sfs.k_feature_idx_)]

lin.fit(X_train[columns], y_train)
y_pred = lin.predict(X_test[columns])
r2_score(y_test, y_pred)

0.7239540198372378

In [ ]:
columns

Index(['MSSubClass', 'LotArea', 'OverallQual', 'MasVnrArea', 'ExterQual',
       'BsmtExposure', 'BsmtFinSF1', 'GrLivArea', 'KitchenQual', 'GarageCars'],
      dtype='object')

Test with 10 highest correlated features

In [ ]:
corr_matrix['SalePrice'].sort_values(ascending=False)[1:11].index

Index(['OverallQual', 'GrLivArea', 'ExterQual', 'KitchenQual', 'GarageCars',
       'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'BsmtQual', 'FullBath'],
      dtype='object')

In [ ]:
columns = corr_matrix['SalePrice'].sort_values(ascending=False)[1:11].index

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('SalePrice', axis=1), y, random_state=0)

lin = LinearRegression()
lin.fit(X_train[columns], y_train)
y_pred = lin.predict(X_test[columns])
r2_score(y_test, y_pred)

0.7028207002269188

**Penggunaan fitur dengan korelasi yang tinggi tidak menjamin hasil prediksi yang baik, tetapi dengan menggunakan fitur yang terbaik(relevan) akan menghasilkan nilai prediksi yang baik**

**Fitur Terbaik :**
1.  MSSubClass
2. LotArea
2. OverallQual
2. MasVnrArea
2. ExterQual
2. BsmtExposure
2. BsmtFinSF1
2. GrLivArea
2. KitchenQual
2. GarageCar


**Korelasi Tertinggi :**
1. 1stFlrSF
1. TotRmsAbvGrd
1. FireplaceQu
1. GarageArea
1. GarageQual
1. GarageCond

